In [1]:
# Karly July 11

# Phase 1 - similarityCompetition
# Analysts tend to use different descriptions to refer to the same security. 
# In this challenge, you are asked to make use of these descriptions to predict
# whether a pair of descriptions refers to the same security or not.

In [60]:
import pandas as pd
import numpy as np
#import nltk
#import tensorflow
from nltk.metrics import edit_distance  

In [61]:
# The "train.csv" = your training dataset
with open('train.csv') as f:
    Data1 = pd.read_csv(f, index_col=0, encoding='utf-8')
with open('train.csv') as f:
    Data2 = pd.read_csv(f, index_col=0, encoding='utf-8')
    
#with open('Kaggletestfile.csv') as f:
#    Data1 = pd.read_csv(f, index_col=0, encoding='utf-8')
    
# The "test.csv" = your test dataset used to grade your results
with open('test.csv') as f:
    Data1Test = pd.read_csv(f, index_col=0, encoding='utf-8')

In [62]:
Data1.head()

description_x  \
test_id                                                      
0                           first trust dow jones internet   
1                      schwab intl large company index etf   
2                             vanguard small cap index adm   
3        duke energy corp new com new isin #us4 sedol #...   
4                                         visa inc class a   

                                             description_y ticker_x ticker_y  \
test_id                                                                        
0                              first trust dj internet idx      FDN      FDN   
1        schwab strategic tr fundamental intl large co ...     FNDF     FNDF   
2                       vanguard small-cap index fund inst    VSMAX    VSCIX   
3        duke energy corp new com new isin #us26441c204...      DUK      DUK   
4                                                visa inc.        V        V   

         same_security  
test_id                 
0                 True  
1                 True  
2                False  
3                 True  
4                 True

In [63]:
Data1Test.head()

description_x                     description_y  \
test_id                                                                         
0                              semtech corp               semtech corporation   
1                    vanguard mid cap index         vanguard midcap index - a   
2               spdr gold trust gold shares  spdr gold trust spdr gold shares   
3             vanguard total bond index adm  vanguard total bond market index   
4        oakmark international fund class i        oakmark international cl i   

         same_security  
test_id                 
0                  NaN  
1                  NaN  
2                  NaN  
3                  NaN  
4                  NaN

In [64]:
# Testing to make sure the column same_security is ok   
Data1['testingTickersCol'] = np.where(Data1['ticker_x'] == Data1['ticker_y'], True, False)
Data1['testingSameSecurityCol'] = np.where(Data1['same_security'] == Data1['testingTickersCol'], 'True', 'False')
Data1[Data1.testingSameSecurityCol == 'False'].shape[0]
# It is valid
Data1 = Data1.drop(['testingTickersCol', 'testingSameSecurityCol'], axis=1)

In [65]:
# Note right now the criteria value is 40 -> I can change this
Data1["EditDistance"] = Data1.apply(lambda x: edit_distance(x["description_x"], x["description_y"]), axis=1)
Data1['testing1'] = np.where(Data1['EditDistance'] < 40, True, False)

Data1['testing'] = np.where(Data1['testing1'] == Data1['same_security'], 'True', 'False')
print("Incorrect Guesses Using Edit Distance:", Data1[Data1.testing == 'False'].shape[0])
print("Correct Guesses Using Edit Distance:", Data1[Data1.testing == 'True'].shape[0])

#Data1
Data1 = Data1.drop(['testing', 'testing1'], axis = 1)

Incorrect Guesses Using Edit Distance: 537
Correct Guesses Using Edit Distance: 1605


In [66]:
Data1.sort_values(['EditDistance'], ascending = [False]).head()

description_x  \
test_id                                                      
484                          vanguard small cap growth etf   
775                      vanguard ftse all-wld ex-u.s.smcp   
799      vanguard index fds formerly vanguard index tr ...   
868          companhia d snmnto bsco d estdo sao paulo adr   
1800     vanguard index fds vanguard small cap vipers f...   

                                             description_y ticker_x ticker_y  \
test_id                                                                        
484      vanguard index fds vanguard small cap growth v...      VBK      VBK   
775      vanguard intl equity index fdsftse all world e...      VSS      VSS   
799                         vanguard index fds s&p 500 etf      VOO      VOO   
868      cia saneamento basico de sao paulo spons adr e...      SBS      SBS   
1800                      vanguard crsp us small cap index       VB       VB   

         same_security  EditDistance  
test_id                               
484               True            50  
775               True            50  
799               True            49  
868               True            45  
1800              True            44

In [67]:
# https://www.python-course.eu/levenshtein_distance.php
def iterative_levenshtein(s, t, costs=(1, 1, 1)):
    """ 
        iterative_levenshtein(s, t) -> ldist
        ldist is the Levenshtein distance between the strings 
        s and t.
        For all i and j, dist[i,j] will contain the Levenshtein 
        distance between the first i characters of s and the 
        first j characters of t
        
        costs: a tuple or a list with three integers (d, i, s)
               where d defines the costs for a deletion
                     i defines the costs for an insertion and
                     s defines the costs for a substitution
    """
    rows = len(s)+1
    cols = len(t)+1
    deletes, inserts, substitutes = costs
    
    dist = [[0 for x in range(cols)] for x in range(rows)]
    # source prefixes can be transformed into empty strings 
    # by deletions:
    for row in range(1, rows):
        dist[row][0] = row * deletes
    # target prefixes can be created from an empty source string
    # by inserting the characters
    for col in range(1, cols):
        dist[0][col] = col * inserts
        
    for col in range(1, cols):
        for row in range(1, rows):
            if s[row-1] == t[col-1]:
                cost = 0
            else:
                cost = substitutes
            dist[row][col] = min(dist[row-1][col] + deletes,
                                 dist[row][col-1] + inserts,
                                 dist[row-1][col-1] + cost) # substitution
    return dist[row][col]

In [68]:
# making substitutions more costly than inserts and deletes
# Note right now the criteria value is 35 -> I can change this
Data1["EditDistanceSubWorse"] = Data1.apply(lambda x: iterative_levenshtein(x["description_x"], x["description_y"], costs = (1, 1, 2)), axis=1)
Data1['testing1'] = np.where(Data1['EditDistanceSubWorse'] < 35, True, False)

Data1['testing'] = np.where(Data1['testing1'] == Data1['same_security'], 'True', 'False')
print("Incorrect Guesses Using Edit Distance With Substitutions Counting more:", Data1[Data1.testing == 'False'].shape[0])
print("Correct Guesses Using Edit Distance With Substitutions Counting more:", Data1[Data1.testing == 'True'].shape[0])

#Data1
Data1 = Data1.drop(['testing', 'testing1'], axis = 1)

Incorrect Guesses Using Edit Distance With Substitutions Counting more: 562
Correct Guesses Using Edit Distance With Substitutions Counting more: 1580


In [69]:
# Data1.sort_values(['EditDistanceSubWorse'], ascending = [False])

In [70]:
# Data1.sort_values(['same_security'], ascending = [False])

In [71]:
import jellyfish
from jellyfish import match_rating_comparison
from jellyfish import jaro_winkler

In [72]:
Data1["MatchRatingComparison"] = Data1.apply(lambda x: match_rating_comparison(x["description_x"], x["description_y"]), axis=1)
Data1['testing'] = np.where(Data1['same_security'] == Data1['MatchRatingComparison'], 'True', 'False')
print("Incorrect Guesses Using Match Rating Comparison:", Data1[Data1.testing == 'False'].shape[0])
print("Correct Guesses Using Match Rating Comparison:", Data1[Data1.testing == 'True'].shape[0])

Data1 = Data1.drop(['testing'], axis = 1)

Incorrect Guesses Using Match Rating Comparison: 521
Correct Guesses Using Match Rating Comparison: 1621


In [73]:
# 0 represents two completely dissimilar strings and 1 represents identical strings
# Note right now the criteria value is 0.7 -> I can change this

# http://jellyfish.readthedocs.io/en/latest/comparison.html#match-rating-approach-comparison
Data1["jaroWinkler"] = Data1.apply(lambda x: jaro_winkler(x["description_x"], x["description_y"]), axis=1)
Data1['testing1'] = np.where(Data1['jaroWinkler']>=0.70, True, False)

Data1['testing'] = np.where(Data1['testing1'] == Data1['same_security'], 'True', 'False')
print("Incorrect Guesses Using Jaro Winkler:", Data1[Data1.testing == 'False'].shape[0])
print("Correct Guesses Using Jaro Winkler:", Data1[Data1.testing == 'True'].shape[0])

#Data1
Data1 = Data1.drop(['testing', 'testing1'], axis = 1)

Incorrect Guesses Using Jaro Winkler: 490
Correct Guesses Using Jaro Winkler: 1652


In [74]:
import nltk, string
from sklearn.feature_extraction.text import TfidfVectorizer

#nltk.download('punkt') # if necessary...


stemmer = nltk.stem.porter.PorterStemmer()
remove_punctuation_map = dict((ord(char), None) for char in string.punctuation)

def stem_tokens(tokens):
    return [stemmer.stem(item) for item in tokens]

'''remove punctuation, lowercase, stem'''
def normalize(text):
    return stem_tokens(nltk.word_tokenize(text.lower().translate(remove_punctuation_map)))

vectorizer = TfidfVectorizer(tokenizer=normalize, stop_words='english')

def cosine_sim(text1, text2):
    tfidf = vectorizer.fit_transform([text1, text2])
    return ((tfidf * tfidf.T).A)[0,1]

Data1["cosineSim"] = Data1.apply(lambda x: cosine_sim(x["description_x"], x["description_y"]), axis=1)
Data1['testing1'] = np.where(Data1['cosineSim']>=0.1, True, False)

Data1['testing'] = np.where(Data1['testing1'] == Data1['same_security'], 'True', 'False')
print("Incorrect Guesses Using Cosine Similarity:", Data1[Data1.testing == 'False'].shape[0])
print("Correct Guesses Using Cosine Similarity:", Data1[Data1.testing == 'True'].shape[0])

#Data1
Data1 = Data1.drop(['testing', 'testing1'], axis = 1)


/Users/ulgk/anaconda/lib/python3.6/site-packages/sklearn/feature_extraction/text.py:1059: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  if hasattr(X, 'dtype') and np.issubdtype(X.dtype, np.float):


Incorrect Guesses Using Cosine Similarity: 499
Correct Guesses Using Cosine Similarity: 1643


In [75]:
Data1.head()

description_x  \
test_id                                                      
0                           first trust dow jones internet   
1                      schwab intl large company index etf   
2                             vanguard small cap index adm   
3        duke energy corp new com new isin #us4 sedol #...   
4                                         visa inc class a   

                                             description_y ticker_x ticker_y  \
test_id                                                                        
0                              first trust dj internet idx      FDN      FDN   
1        schwab strategic tr fundamental intl large co ...     FNDF     FNDF   
2                       vanguard small-cap index fund inst    VSMAX    VSCIX   
3        duke energy corp new com new isin #us26441c204...      DUK      DUK   
4                                                visa inc.        V        V   

         same_security  EditDistance  EditDistanceSubWorse  \
test_id                                                      
0                 True            11                    11   
1                 True            30                    30   
2                False             9                    12   
3                 True            11                    11   
4                 True             8                     9   

         MatchRatingComparison  jaroWinkler  cosineSim  
test_id                                                 
0                         True     0.896111   0.336097  
1                         True     0.835844   0.572739  
2                         True     0.892502   0.252334  
3                         True     0.942637   0.716812  
4                         True     0.877778   0.579739

In [76]:
Data1[Data1.same_security == False].shape[0]

529

In [79]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os

import spacy                   #Python package for NLP
#import Levenshtein as lv       #To compute levenshtein distance
from sklearn.preprocessing import StandardScaler       #Used to normalize data (substract the mean and divide by std)
from sklearn.linear_model import LogisticRegression      #Logistic regression classifier
#from difflib import SequenceMatcher

nlp = spacy.load('en')    #Loading english language models in spacy

#df_train = Data1  #Loading training data
#df_test = Data1Test   #Loading test data

msk = np.random.rand(len(Data1)) < 0.8
df_train = Data2[msk]
df_test = Data2[~msk]


def get_features(str1, str2):
    """
    This function computes some features from a pair of strings. It is an auxiliary function that will be used to generate a
    feature represenation for each example in the data. In this sample code I am computing 6 features (many more could be added):
    - difference in length in both strings
    - levenshtein ratio between the two strings
    - levenshtein distance between the two strings
    - difference in number of words
    - similarity between the two strings using spacy
    - difference in length after removing stop words from both strings
    """
    feats = {}
    feats['diff_len'] = len(str1) - len(str2)  #adding difference in len as a feature
    
   # feats['lev_ratio'] = lv.ratio(str1, str2)   # adding levenshtein ratio
    #feats['lv_dist'] = lv.distance(str1, str2)  # adding levenshtein distance
    feats['lv_dist'] = iterative_levenshtein(str1, str2, costs = (1, 1, 1)) # adding levenshtein distance
    feats['edit_distance'] = edit_distance(str1, str2)
    feats['match_rating'] = match_rating_comparison(str1, str2)
    feats['jaroWinkler'] = jaro_winkler(str1, str2)
    feats['cosineSim'] = cosine_sim(str1, str2)
    
    #s = SequenceMatcher(None, str1, str2)
    #feats['sequenceMatch'] = s.ratio()
    
    feats['jarodist'] = jellyfish.jaro_distance(str1, str2)
    feats['dameraudist'] = jellyfish.damerau_levenshtein_distance(str1, str2)
    
    s1 = nlp(str1)
    s2 = nlp(str2)

    feats['diff_words'] = len(s1) - len(s2)    #adding difference in number of words as a feature
    
    feats['spacy_sim'] = s1.similarity(s2)    #adding similarity between strings from spacy
    
    no_stop_s1 = [w for w in s1 if not w.is_stop]
    no_stop_s2 = [w for w in s2 if not w.is_stop]
    feats['diff_no_stop'] = len(no_stop_s1) - len(no_stop_s2)  #adding the difference in len after removing stop words
    

    if ("inv" in str1 and "adm" in str2) or ("adm" in str1 and "inv" in str2):
        feats['adminv'] = 1
    else:
        feats['adminv'] = 0
        
  #  if ("cl" in str1 and "cl" not in str2) or ("cl" in str2 and "cl" not in str1):
  #      feats['clfeat'] = 1
  #  else:
  #      feats['clfeat'] = 0
        
  #  if ("adm" in str1 and "adm" not in str2) or ("adm" in str2 and "adm" not in str1):
  #      feats['admfeat'] = 1
  #  else:
  #      feats['admfeat'] = 0
        
    if ("inv" in str1 and "ins" in str2) or ("ins" in str1 and "inv" in str2):
        feats['insinv'] = 1
    else:
        feats['insinv'] = 0
        
    if ("cl" in str1 and "ins" in str2) or ("ins" in str1 and "cl" in str2):
        feats['inscl'] = 1
    else:
        feats['inscl'] = 0
        
    if ("adm" in str1 and "ins" in str2) or ("ins" in str1 and "adm" in str2):
        feats['insadm'] = 1
    else:
        feats['insadm'] = 0
        
    if ("cl" in str1 and "adm" in str2) or ("adm" in str1 and "cl" in str2):
        feats['admcl'] = 1
    else:
        feats['admcl'] = 0
        
    if ("index" in str1 and "index" not in str2) or ("index" in str2 and "index" not in str1):
        feats['indexfeat'] = 1
    else:
        feats['indexfeat'] = 0

    
    return feats


def feature_engineering(df):
    """
    Given a data frame with 'description_x' and 'description_y', it will apply 'get_features' function to all rows to compute the matrix 
    with the data representation. This will be applied to df_train and df_test before training the model. 
    """
    list_dicts = df.apply(lambda x: get_features(x['description_x'], x['description_y']), axis=1).values.tolist()
    df_feats = pd.DataFrame(list_dicts)
        
    print(df_feats)
    return df_feats.values 


X_train = feature_engineering(df_train)    #training set data
y_train = df_train.same_security.values    #training set labels

X_test = feature_engineering(df_test)      #test set

#We should normalize features before applying a logistic regression algorithm 
ss = StandardScaler()  
X_train = ss.fit_transform(X_train)
X_test = ss.transform(X_test)

#Applying logistic regresion classifier
lr = LogisticRegression()  #Using default parameters.
lr.fit(X_train, y_train)    #training the model with X_train, y_train
y_pred = lr.predict_proba(X_test)[:,1]  #making predictions on the test set and getting probabilities of the prediction to be equal to 1.

#generating the submission file.
df_sub = df_test.copy()
df_sub['same_security_actual'] = df_sub['same_security']
df_sub['same_security'] = y_pred
df_sub.loc[:, ['same_security']].to_csv('logistic_regression.csv', index=True, index_label='id')


/Users/ulgk/anaconda/lib/python3.6/site-packages/sklearn/feature_extraction/text.py:1059: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  if hasattr(X, 'dtype') and np.issubdtype(X.dtype, np.float):


      admcl  adminv  cosineSim  dameraudist  diff_len  diff_no_stop  \
0         0       0   0.336097           11         3             0   
1         0       0   0.572739           30       -20            -3   
2         0       0   0.252334            9        -6            -2   
3         0       0   0.716812           11        -7             0   
4         0       0   0.579739            8         7             0   
5         0       0   0.502329           15        15             4   
6         0       0   0.127360           12         1            -3   
7         0       0   0.716812            9         9             0   
8         0       0   0.127360           19         5             1   
9         0       0   0.602975            2        -2             0   
10        0       0   0.127360           15        -4            -1   
11        0       0   0.709297            8        -8            -2   
12        0       0   0.510149           11        -9            -1   
13    

     admcl  adminv  cosineSim  dameraudist  diff_len  diff_no_stop  \
0        0       0   0.184432           22         8             1   
1        0       0   0.669419            5         3             0   
2        0       0   0.356300            8         6             2   
3        0       0   0.380873           22        -4            -1   
4        0       0   0.225765           10         1             0   
5        0       0   0.336010           35       -27            -2   
6        0       0   0.115216           22        19             2   
7        0       0   0.602975            5        -5             0   
8        0       1   0.101631           29        24             3   
9        0       0   0.356300           22       -16            -4   
10       0       0   0.411207           11        -9            -1   
11       0       0   0.336097            6        -2             1   
12       0       0   0.503103            8         5            -1   
13       0       0  

/Users/ulgk/anaconda/lib/python3.6/site-packages/sklearn/utils/validation.py:429: DataConversionWarning: Data with input dtype object was converted to float64 by StandardScaler.
  warnings.warn(msg, _DataConversionWarning)


In [80]:
df_sub['same_security_check'] = np.where(df_sub['same_security'] > 0.4, True, False)
df_sub.head()
df_sub['testing'] = np.where(df_sub['same_security_check'] == df_sub['same_security_actual'], 'True', 'False')
print("Incorrect Guesses:", df_sub[df_sub.testing == 'False'].shape[0])
print("Correct Guesses:", df_sub[df_sub.testing == 'True'].shape[0])
print("Accuracy:", df_sub[df_sub.testing == 'True'].shape[0] / df_sub.shape[0])
print("_________0.4 above____________________________")

df_sub['same_security_check'] = np.where(df_sub['same_security'] > 0.5, True, False)
df_sub.head()
df_sub['testing'] = np.where(df_sub['same_security_check'] == df_sub['same_security_actual'], 'True', 'False')
print("Incorrect Guesses:", df_sub[df_sub.testing == 'False'].shape[0])
print("Correct Guesses:", df_sub[df_sub.testing == 'True'].shape[0])
print("Accuracy:", df_sub[df_sub.testing == 'True'].shape[0] / df_sub.shape[0])
print("___________0.5 above__________________________")

df_sub['same_security_check'] = np.where(df_sub['same_security'] > 0.55, True, False)
df_sub.head()
df_sub['testing'] = np.where(df_sub['same_security_check'] == df_sub['same_security_actual'], 'True', 'False')
print("Incorrect Guesses:", df_sub[df_sub.testing == 'False'].shape[0])
print("Correct Guesses:", df_sub[df_sub.testing == 'True'].shape[0])
print("Accuracy:", df_sub[df_sub.testing == 'True'].shape[0] / df_sub.shape[0])
print("____________0.55 above_________________________")

df_sub['same_security_check'] = np.where(df_sub['same_security'] > 0.6, True, False)
df_sub.head()
df_sub['testing'] = np.where(df_sub['same_security_check'] == df_sub['same_security_actual'], 'True', 'False')
print("Incorrect Guesses:", df_sub[df_sub.testing == 'False'].shape[0])
print("Correct Guesses:", df_sub[df_sub.testing == 'True'].shape[0])
print("Accuracy:", df_sub[df_sub.testing == 'True'].shape[0] / df_sub.shape[0])
print("___________0.6__________________________")

df_sub['same_security_check'] = np.where(df_sub['same_security'] > 0.7, True, False)
df_sub.head()
df_sub['testing'] = np.where(df_sub['same_security_check'] == df_sub['same_security_actual'], 'True', 'False')
print("Incorrect Guesses:", df_sub[df_sub.testing == 'False'].shape[0])
print("Correct Guesses:", df_sub[df_sub.testing == 'True'].shape[0])
print("Accuracy:", df_sub[df_sub.testing == 'True'].shape[0] / df_sub.shape[0])
print("_____________0.7 above________________________")

# using everything with a 0.5 threshold gives 49 incorrect
# using everything but index does a lot worse, 56 correct at 0.55
# removing lv_dist, match_rating and diff_words: 53 correct at 0.6
# removing cl gives 48 incorrect at 0.4
# removing cl and adm gives 39 incorrect at 0.5
# removing cl adm and index gives 48 incorrect at 0.5
# removing cl adm and cosinedist gives 54 at 0.5


Incorrect Guesses: 47
Correct Guesses: 325
Accuracy: 0.8736559139784946
_________0.4 above____________________________
Incorrect Guesses: 48
Correct Guesses: 324
Accuracy: 0.8709677419354839
___________0.5 above__________________________
Incorrect Guesses: 50
Correct Guesses: 322
Accuracy: 0.8655913978494624
____________0.55 above_________________________
Incorrect Guesses: 55
Correct Guesses: 317
Accuracy: 0.8521505376344086
___________0.6__________________________
Incorrect Guesses: 54
Correct Guesses: 318
Accuracy: 0.8548387096774194
_____________0.7 above________________________


In [21]:
df_sub[df_sub.testing == 'False']

description_x  \
test_id                                                      
34                                pimco total ret fd instl   
59                   metropolitan west total return bond m   
78                                  vanguard growth etf iv   
81                         aqr managed futures strategy r6   
101                           pimco total return fund cl p   
132             vanguard mid-cap index fund admiral shares   
135                                        vg reit idx etf   
137                                    trp retirement 2050   
170                        guggenheim mid cap val fd instl   
173                                      fidelity balanced   
247                                   af new perspective a   
253                                           at&t inc com   
278                   american funds europacific growth-r5   
327                           metropolitan west totret bdm   
341                           pimco total return fund cl p   
420                                    vang selected value   
467                            loomis sayles bond instl cl   
474                    oppenheimer interntl bond fund cl i   
499                       pimco total return fund instl cl   
514                        aqr managed futures strategy r6   
659                                      msft common stock   
666                                     google inc class a   
713                                 pimco income fund cl p   
718                                activision blizzard inc   
766                             pimco total return fd cl c   
904                         transocean limited com chf0.10   
914                        aqr managed futures strategy r6   
962                           t rowe price small cap stock   
964                              alphabet inc cap stk cl a   
1015                      washington mutual investors fund   
...                                                    ...   
1094                metropolitan west tot return bond cl m   
1105                metropolitan west tot return bond cl m   
1157                                 alphabet inc shs cl c   
1165                            general motors company com   
1177                      metropolitan west total return m   
1209                             am fnd europacfic grth r6   
1287                          blackrock strategic income i   
1372     vanguard intermediate-term tax-exempt fund adm...   
1412                                      fa real estate i   
1435                           amer funds europac r3 m4740   
1480                           c.h. robinson worldwide inc   
1484                                  amgen inc div: 4.000   
1490     rydex guggenheim s&p 500 equal weighted index etf   
1511     vanguard total bond market index fund institut...   
1569     vanguard intl equity index fds ftse emerging m...   
1583                 metropolitan west total return bond m   
1652           royal dutch shell adr ea rep 2 cl a eur0.07   
1713     growth fund of america class a - american fund...   
1752                          metropolitan west totret bdm   
1783                harris oakmark international fund cl i   
1799     vanguard short-term bond index fund admiral sh...   
1803                         t. rowe price health sciences   
1844                                    s&p 500 index spdr   
1898                      american funds europacific gr r4   
1920                   sprtn total mkt indx investor class   
1924                        metropolitan west total return   
1927                            pimco total return fd cl c   
2015                    vanguard total bond market index i   
2040                         templeton global bond class a   
2058              fidelity msci consumer staples index etf   

                                             description_y ticker_x ticker_y  \
test_id                                                   

In [82]:
import numpy as np 
import pandas as pd 
import os
import spacy           
from sklearn.preprocessing import StandardScaler       
from sklearn.linear_model import LogisticRegression      

nlp = spacy.load('en')    #Loading english language models in spacy

df_train = Data1  #Loading training data
df_test = Data1Test[['description_x', 'description_y']]   #Loading test data # df[['a','b']]

#msk = np.random.rand(len(Data1)) < 0.8
#df_train = Data2[msk]
#df_test = Data2[~msk]


def get_features(str1, str2):
    feats = {}
    feats['diff_len'] = len(str1) - len(str2)  
    feats['lv_dist'] = iterative_levenshtein(str1, str2, costs = (1, 1, 1)) 
    feats['edit_distance'] = edit_distance(str1, str2)
    #feats['match_rating'] = match_rating_comparison(str1, str2)
    feats['jaroWinkler'] = jaro_winkler(str1, str2)
    feats['cosineSim'] = cosine_sim(str1, str2)
    feats['jarodist'] = jellyfish.jaro_distance(str1, str2)
    feats['dameraudist'] = jellyfish.damerau_levenshtein_distance(str1, str2)
    
    s1 = nlp(str1)
    s2 = nlp(str2)

    feats['diff_words'] = len(s1) - len(s2)    
    feats['spacy_sim'] = s1.similarity(s2)    
    
    no_stop_s1 = [w for w in s1 if not w.is_stop]
    no_stop_s2 = [w for w in s2 if not w.is_stop]
    feats['diff_no_stop'] = len(no_stop_s1) - len(no_stop_s2)  
    

    if ("inv" in str1 and "adm" in str2) or ("adm" in str1 and "inv" in str2):
        feats['adminv'] = 1
    else:
        feats['adminv'] = 0
        
  #  if ("cl" in str1 and "cl" not in str2) or ("cl" in str2 and "cl" not in str1):
  #      feats['clfeat'] = 1
  #  else:
  #      feats['clfeat'] = 0
        
  #  if ("adm" in str1 and "adm" not in str2) or ("adm" in str2 and "adm" not in str1):
  #      feats['admfeat'] = 1
  #  else:
  #      feats['admfeat'] = 0
        
    if ("inv" in str1 and "ins" in str2) or ("ins" in str1 and "inv" in str2):
        feats['insinv'] = 1
    else:
        feats['insinv'] = 0
        
    if ("cl" in str1 and "ins" in str2) or ("ins" in str1 and "cl" in str2):
        feats['inscl'] = 1
    else:
        feats['inscl'] = 0
        
    if ("adm" in str1 and "ins" in str2) or ("ins" in str1 and "adm" in str2):
        feats['insadm'] = 1
    else:
        feats['insadm'] = 0
        
    if ("cl" in str1 and "adm" in str2) or ("adm" in str1 and "cl" in str2):
        feats['admcl'] = 1
    else:
        feats['admcl'] = 0
        
    if ("index" in str1 and "index" not in str2) or ("index" in str2 and "index" not in str1):
        feats['indexfeat'] = 1
    else:
        feats['indexfeat'] = 0

    
    return feats


def feature_engineering(df):
    list_dicts = df.apply(lambda x: get_features(x['description_x'], x['description_y']), axis=1).values.tolist()
    df_feats = pd.DataFrame(list_dicts)
        
    #print(df_feats)
    return df_feats.values 


X_train = feature_engineering(df_train)    #training set data
y_train = df_train.same_security.values    #training set labels

X_test = feature_engineering(df_test)      #test set

# Normalize features before applying a logistic regression algorithm 
ss = StandardScaler()  
X_train = ss.fit_transform(X_train)
#print(X_test)
#X_test = X_test.replace(True, X_test.replace([True], [None])
X_test = ss.transform(X_test)

# logistic regresion classifier
lr = LogisticRegression() 
lr.fit(X_train, y_train)   
y_pred = lr.predict_proba(X_test)[:,1]  

#generating the submission file.
df_sub = df_test.copy()
#df_sub['same_security_actual'] = df_sub['same_security']
#df_sub['same_security_pred'] = y_pred
#df_sub['same_security'] = np.where(df_sub['same_security_pred'] > 0.5, True, False)

df_sub['same_security'] = y_pred
df_sub['same_security'] = np.where(df_sub['same_security'] > 0.5, True, False)
df_sub = df_sub[['description_x', 'description_y', 'same_security']]

df_sub.to_csv('submission_phase1_abracaData.csv', index=True, index_label='id')

#df_sub.loc[:, [['same_security']].to_csv('submission_phase1_abracaData.csv', index=True, index_label='id')


/Users/ulgk/anaconda/lib/python3.6/site-packages/sklearn/feature_extraction/text.py:1059: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  if hasattr(X, 'dtype') and np.issubdtype(X.dtype, np.float):


In [35]:
len(X_test)
X_test_df = pd.DataFrame(X_test)
len(X_test_df)
len(X_test_df.dropna())
pd.options.display.max_rows = 999